# Rescale optical depth and compute 1D power spectrum

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import extract_skewers
import read_gadget
import fake_spectra.tempdens as tdr
import fake_spectra.griddedspectra as grid_spec

### Read a single snapshot, look at different temperatures 

In [2]:
basedir='/Users/font/CosmoData/test_sim_emulator/test_march_2019/'
skewers_dir=basedir+'/output/skewers/'
test_snap_num=8
n_skewers=10
width_Mpc=0.1

In [3]:
# list of temperature scalings
scales_T0=[0.8,1.0,1.2]
#scales_gamma=[0.8,1.0,1.2]
scales_gamma=[1.0]

In [5]:
for scale_T0 in scales_T0:
    for scale_gamma in scales_gamma:
        sk_filename=extract_skewers.get_skewers_filename(test_snap_num,n_skewers,width_Mpc,scale_T0,scale_gamma)
        print(sk_filename)

skewers_8_Ns10_wM0.1_sT0.8_sg1.0.hdf5
skewers_8_Ns10_wM0.1_sT1.0_sg1.0.hdf5
skewers_8_Ns10_wM0.1_sT1.2_sg1.0.hdf5
